In [1]:
import numpy as np
import scipy as sp

import matplotlib.pyplot as plt
import matplotlib.axes as axe
import pandas as pd
import datetime as dt
import gurobipy as gp
from gurobipy import GRB
import cvxpy as cp

import random
from itertools import chain, combinations, tee
import time


# Functions

In [2]:
def demand_name_by_group_index(index):
    list_demand_names = ["Demand (eligible group, 1)", "Demand (eligible group, 2)", \
                         "Demand (ineligible group, 1)", "Demand (ineligible group, 2)", \
                         "Demand (ineligible group, 3)"]
    return list_demand_names[index]

def VoT_name_by_group_index(index):
    list_demand_names = ["VoT (eligible group, 1)", "VoT (eligible group, 2)", \
                         "VoT (ineligible group, 1)", "VoT (ineligible group, 2)", \
                         "VoT (ineligible group, 3)"]
    return list_demand_names[index]


# Groups, Routes to Edges:

In [3]:
directory_path = '../data/data_income_percentage_VoT/'
df_data = pd.read_csv(directory_path + 'data_cities_od_VoTs_demands_1.csv')

# df_od_flow_data
# df_data

In [4]:
dict_data = {}

for column_name_full in list(df_data.columns):
    if column_name_full == "Data Category":
        categories_list = df_data[column_name_full].tolist()
    else:
        dict_data[int(column_name_full)] = {}
        for category_index, category in enumerate(categories_list):
            if category == "Start City Index" or category == "End City Index":
                dict_data[int(column_name_full)][category] \
                    = int(df_data[column_name_full].tolist()[category_index])
            elif category == "Start City" or category == "End City":
                dict_data[int(column_name_full)][category] \
                    = df_data[column_name_full].tolist()[category_index]
            else:
#                 print("category:", category)
                dict_data[int(column_name_full)][category] \
                    = float(df_data[column_name_full].tolist()[category_index])


In [5]:
dict_data

{0: {'Start City Index': 0,
  'End City Index': 0,
  'Start City': 'Palo Alto',
  'End City': 'Palo Alto',
  'O-D Flow (Max Entropy)': 617.3087999,
  'Demand (eligible group, 1)': 48.15008639,
  'VoT (eligible group, 1)': 0.031974236,
  'Demand (eligible group, 2)': 27.778896,
  'VoT (eligible group, 2)': 0.10238604,
  'Demand (ineligible group, 1)': 116.0540544,
  'VoT (ineligible group, 1)': 0.275440705,
  'Demand (ineligible group, 2)': 135.807936,
  'VoT (ineligible group, 2)': 0.580929487,
  'Demand (ineligible group, 3)': 289.5178272,
  'VoT (ineligible group, 3)': 1.859644942},
 1: {'Start City Index': 0,
  'End City Index': 1,
  'Start City': 'Palo Alto',
  'End City': 'East Palo Alto',
  'O-D Flow (Max Entropy)': 92.22995123,
  'Demand (eligible group, 1)': 7.193936196,
  'VoT (eligible group, 1)': 0.031974236,
  'Demand (eligible group, 2)': 4.150347805,
  'VoT (eligible group, 2)': 0.10238604,
  'Demand (ineligible group, 1)': 17.33923083,
  'VoT (ineligible group, 1)': 0.27

In [6]:
cities_dict = {}
for od_info in list(dict_data.values()):
    if od_info["Start City Index"] not in list(cities_dict.keys()):
        cities_dict[od_info["Start City Index"]] = od_info["Start City"]
    if od_info["End City Index"] not in list(cities_dict.keys()):
        cities_dict[od_info["End City Index"]] = od_info["End City"]

cities_list = list(cities_dict.values())

# cities_dict

In [7]:
od_to_edges_array = np.zeros((len(list(dict_data.keys())), 2))

for od_index, od_info in dict_data.items():
    od_to_edges_array[od_index, 0] = int(cities_list.index(od_info["Start City"]))
    od_to_edges_array[od_index, 1] = int(cities_list.index(od_info["End City"]))

# od_to_edges_array

In [8]:
num_groups_per_od = 5

demand_array = np.zeros((len(list(dict_data.keys())), num_groups_per_od))
VoT_array = np.zeros((len(list(dict_data.keys())), num_groups_per_od))

for od_index, od_value in dict_data.items():
    for group_index in range(num_groups_per_od):
        demand_name = demand_name_by_group_index(group_index)
        VoT_name = VoT_name_by_group_index(group_index)
        
        demand_array[od_index, group_index] = od_value[demand_name]
        VoT_array[od_index, group_index] = od_value[VoT_name]

# print(demand_array)
# VoT_array

# Download Latency Parameters Data

In [9]:
directory_path_latency = '../data/pems_latency_inference/'
df_latency_params = pd.read_csv(directory_path_latency + 'latency_params.csv')

# list(df_latency_params.loc[:, "Palo Alto"])

In [10]:
dict_latency_params = {}

city_list = list(df_latency_params.columns)[1:]

for city in city_list:
    if city != "Belmont":
        dict_latency_params[city] = {}
        dict_latency_params[city]["Flow (at bend)"] = df_latency_params.loc[:, city][0]
        dict_latency_params[city]["Time (at bend)"] = df_latency_params.loc[:, city][1]
        dict_latency_params[city]["Slope (after bend)"] = df_latency_params.loc[:, city][2]

dict_latency_params

{'Palo Alto': {'Flow (at bend)': 811.224,
  'Time (at bend)': 0.02194693,
  'Slope (after bend)': 9.21e-06},
 'East Palo Alto': {'Flow (at bend)': 953.2464286,
  'Time (at bend)': 0.036732014,
  'Slope (after bend)': 1.03e-05},
 'Redwood City': {'Flow (at bend)': 1047.5825,
  'Time (at bend)': 0.082540168,
  'Slope (after bend)': 3.03e-05},
 'San Mateo': {'Flow (at bend)': 1238.36663,
  'Time (at bend)': 0.092069984,
  'Slope (after bend)': 5.27e-05},
 'Burlingame': {'Flow (at bend)': 845.15,
  'Time (at bend)': 0.025025091,
  'Slope (after bend)': 4.45e-06},
 'Millbrae': {'Flow (at bend)': 853.1818182,
  'Time (at bend)': 0.039962578,
  'Slope (after bend)': 5.42e-06}}

# General CBCP Equilibrium Solver

## (Special Case) Quartic Polynomial Latency Functions

In [11]:
# grad = np.array([0, 1, 2, 3, 4])
grad = np.array([2, 4, 0, 1, 3])

for id_temp, entry_temp in enumerate(grad):
    print("id_temp, entry_temp:", id_temp, entry_temp)


id_temp, entry_temp: 0 2
id_temp, entry_temp: 1 4
id_temp, entry_temp: 2 0
id_temp, entry_temp: 3 1
id_temp, entry_temp: 4 3


In [12]:
# np.array([1, 2, 3]) * np.array([6, 5, 3])

In [13]:
def coeff_from_input(coeff_input, num_edges, num_gp_lanes):
    # Goal: Differentiate between express and general purpose lanes.
    
    assert np.all(coeff_input >= 0.0), "All entries of coeff_input must be non-negative."
    assert len(coeff_input.shape) == 2, "coeff_input should be a matrix, i.e., 2-D array"
    assert coeff_input.shape[0] == 3, "Latency functions are assumed to be piecewise linear / affine with 3 parameters."
    assert coeff_input.shape[1] == num_edges, "Latency functions should be defined across all edges."
    
    coeff_length = 3
    
    ex_to_gp_multiplier = np.array([1, 1/num_gp_lanes, num_gp_lanes]).reshape((coeff_length, 1)) \
                            @ np.ones((1, num_edges))
    
    coeff_full = np.zeros((coeff_length, num_edges, 2))
    coeff_full[:, :, 0] = coeff_input
    coeff_full[:, :, 1] = coeff_input * ex_to_gp_multiplier
    
    return coeff_full

def latency_max(flow_max, coeff):
    
    assert np.all(coeff >= 0.0), "coeff should be non-negative"
    assert len(coeff.shape) == 1, "coeff should be a 1-D array."
    assert coeff.shape[0] == 3, "Latency functions are assumed to be piecewise linear / affine with 3 parameters."
    
    return coeff[0] + max(coeff[1] * (flow_max - coeff[2]), 0)

def welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, tau, \
                demand_array, VoT_array, num_el, od_to_edges_array, y, \
                coeff_input):

    coeff = coeff_from_input(coeff_input, num_edges, num_gp_lanes)
    
    assert len(od_to_edges_array.shape) == 2, "od_to_edges should be 2-dimensional."
    assert od_to_edges_array.shape[1] == 2, "od_to_edges' second dimension should be for start and end edges."
    
    edge_to_od_dict = {}
    for e in range(num_edges):
        edge_to_od_dict[e] = [k for k in list(range(int(od_to_edges_array.shape[0]) )) \
                               if od_to_edges_array[k, 0] <= e <= od_to_edges_array[k, 1]]

    num_groups = demand_array.shape[1]
    num_in = num_groups - num_el
    assert num_in >= 0, "We must have num_in >= 0."
    
    el_indices = list(range(num_el))
    in_indices = list(range(num_el, num_groups))
    
#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert len(tau.shape) == 2, "tau should be 2-dimensional."
    assert tau.shape[0] == num_edges, "toll vector's first axis length must equal the number of edges."
    assert tau.shape[1] == T, "toll vector's second axis length must equal the time horizon."
    
    ## Compute lane flows:
    
    x = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            x[e, 0, t] += sum(y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] \
                              for g in range(num_groups))
    for e in range(num_edges):
        for t in range(T):
            x[e, 1, t] += sum( (y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)]) for od in edge_to_od_dict[e] \
                              for g in el_indices)
            x[e, 1, t] += sum(y[(od, g, e, 1, t)] for od in edge_to_od_dict[e] \
                              for g in in_indices)
            
    print()
    print("in_indices:", in_indices)
    print("el_indices:", el_indices)
    print()
    
    ## Compute lane latencies:
    
    ell = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            for k in range(2):
                ell[e, k, t] = coeff[0, e, k] + coeff[1, e, k] * max(x[e, k, t] - coeff[2, e, k], 0)

    obj_E = sum( y[(od, g, e, 0, t)] * VoT_array[od, g] * ell[e, 0, t] \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                for g in el_indices for t in range(T) ) \
            + sum( y[(od, g, e, 1, t)] * (VoT_array[od, g] * ell[e, 0, t] + tau[e, t]) \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                for g in el_indices for t in range(T) ) \
            + sum( y[(od, g, e, 2, t)] * VoT_array[od, g] * ell[e, 1, t] \
                  for e in range(num_edges) for od in edge_to_od_dict[e] \
                  for g in el_indices for t in range(T) )
    obj_I = sum( y[(od, g, e, 0, t)] * (VoT_array[od, g] * ell[e, 0, t] + tau[e, t]) \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                for g in in_indices for t in range(T) ) \
            + sum( y[(od, g, e, 1, t)] * VoT_array[od, g] * ell[e, 1, t] \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                  for g in in_indices for t in range(T) )
    obj_R = sum( y[(od, g, e, 0, t)] * tau[e, t] \
                for e in range(num_edges) for od in edge_to_od_dict[e] \
                for g in in_indices for t in range(T) )

    welfare = lambda_E * obj_E - lambda_R * obj_R + lambda_I * obj_I

#     print()
#     print("obj_E:", obj_E)
#     print("obj_R:", obj_R)
#     print("obj_I:", obj_I)
#     print("welfare:", welfare)
#     print()
    
    return welfare, obj_E, obj_R, obj_I


# Latency, total, throughout the entire time horizon. 

def latency_total(T, num_edges, lambda_E, lambda_R, lambda_I, tau, \
                  demand_array, VoT_array, num_el, y, od_to_edges_array, \
                  coeff_input):
    
    coeff = coeff_from_input(coeff_input, num_edges, num_gp_lanes)
    
    assert len(od_to_edges_array.shape) == 2, "od_to_edges should be 2-dimensional."
    assert od_to_edges_array.shape[1] == 2, "od_to_edges' second dimension should be for start and end edges."
    
    edge_to_od_dict = {}
    for edge in range(num_edges):
        edge_to_od_dict[e] = [k for k in list(range(int(od_to_edges_array.shape[0]) )) \
                               if od_to_edges_array[k, 0] <= e <= od_to_edges_array[k, 1]]
    
    # In full:
    # y indices: (od, group, edge, "lane", time)

    num_groups = demand_array.shape[1]
    num_in = num_groups - num_el
    assert num_in >= 0, "We must have num_in >= 0."
    
    el_indices = list(range(num_el))
    in_indices = list(range(num_el, num_groups))
    
#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert len(tau.shape) == 2, "tau should be 2-dimensional."
    assert tau.shape[0] == num_edges, "toll vector's first axis length must equal the number of edges."
    assert tau.shape[1] == T, "toll vector's second axis length must equal the time horizon."
    
    ## Compute lane flows:
    
    x = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            x[e, 0, t] += sum(y[(od, g, e, 0, t)] for od in edge_to_od_dict[e] \
                              for g in range(num_groups))
    for e in range(num_edges):
        for t in range(T):
            x[e, 1, t] += sum( (y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)]) for od in edge_to_od_dict[e] \
                              for g in el_indices)
            x[e, 1, t] += sum(y[(od, g, e, 1, t)] for od in edge_to_od_dict[e] \
                              for g in in_indices)
    
    ## Compute lane latencies:
    
    ell = np.zeros((num_edges, 2, T))
    for e in range(num_edges):
        for t in range(T):
            for k in range(2):
                ell[e, k, t] = coeff[0, e, k] + coeff[1, e, k] * max(x[e, k, t] - coeff[2, e, k], 0)

    latency = sum( x[e, k, t] * ell[e, k, t] \
                for e in range(num_edges) for k in range(2) for t in range(T) )

#     print()
#     print("obj_E:", obj_E)
#     print("obj_R:", obj_R)
#     print("obj_I:", obj_I)
#     print("welfare:", welfare)
#     print()
    
    return latency

In [14]:
# arr = np.arange(5)
# arr.reshape((5, 1))

In [15]:
def proj_tau_B(T, num_edges, tau, B, od_to_edges_list_full, tau_max, B_max = 1.0, B_min = 0.1):

#     print()
#     print("tau.shape[0]:", tau.shape[0])
#     print("num_edges:", num_edges)
#     print()

    assert tau.shape[0] == num_edges, "tau must have length equal to the number of edges."
    assert tau_max.shape[0] == num_edges, "tau_max must have length equal to the number of edges."
    
    tau_feas = cp.Variable((num_edges, T))
#     B_feas = cp.Variable(1)
    
    func = cp.sum_squares(tau_feas - tau)

    objective = cp.Minimize(func)

    constraints = []
    constraints += [tau_feas >= 0.0]
#     constraints += [B_feas >= 0.0]
    constraints += [tau_feas <= tau_max.reshape((num_edges, 1)) * np.ones((1, T))]
#     constraints += [B_feas <= B_max]

#     constraints += [B_feas <= sum(tau_feas[e, t] for e in od_to_edges_list_full[od] for t in range(T)) \
#                     for od in range(len(od_to_edges_list_full))]
    
    prob = cp.Problem(objective, constraints)
    result = prob.solve()

#     print()
#     print("tau_feas.value:", np.round(np.maximum(tau_feas.value, 0.0), decimals=4))
#     print()

    B_max_list = [sum(tau_feas.value[e, t] for e in od_to_edges_list_full[od] for t in range(T)) \
                            for od in range(len(od_to_edges_list_full))]
    B_max_wrt_tau_feas = min([B_max for B_max in B_max_list if B_max >= B_min])
    B_feas = min(B, B_max_wrt_tau_feas)
    
    print()
    print("B:", B)
    print("B_max_wrt_tau_feas:", B_max_wrt_tau_feas)
    print("B_feas:", B_feas)
    print()

    return np.round(np.maximum(tau_feas.value, 0.0), decimals=4), \
            np.round(np.maximum(B_feas, 0.0), decimals=4)
    
#     return tau_feas.value, B_feas



In [16]:
# # OLD def proj_tau_B:

# def proj_tau_B(T, num_edges, tau, B, od_to_edges_list_full, tau_max, B_max = 1.0):

# #     print()
# #     print("tau.shape[0]:", tau.shape[0])
# #     print("num_edges:", num_edges)
# #     print()

#     assert tau.shape[0] == num_edges, "tau must have length equal to the number of edges."
#     assert tau_max.shape[0] == num_edges, "tau_max must have length equal to the number of edges."
    
#     tau_feas = cp.Variable((num_edges, T))
#     B_feas = cp.Variable(1)
    
#     func = cp.sum_squares(tau_feas - tau) + (B_feas - B)**2

#     objective = cp.Minimize(func)

#     constraints = []
#     constraints += [tau_feas >= 0.0]
#     constraints += [B_feas >= 0.0]
#     constraints += [tau_feas <= tau_max.reshape((num_edges, 1)) * np.ones((1, T))]
#     constraints += [B_feas <= B_max]

#     constraints += [B_feas <= sum(tau_feas[e, t] for e in od_to_edges_list_full[od]) \
#                     for od in range(len(od_to_edges_list_full)) for t in range(T)]
    
#     prob = cp.Problem(objective, constraints)
#     result = prob.solve()

# #     print()
# #     print("tau_feas.value:", np.round(np.maximum(tau_feas.value, 0.0), decimals=4))
# #     print()

#     return np.round(np.maximum(tau_feas.value, 0.0), decimals=4), \
#             np.round(np.maximum(B_feas.value, 0.0), decimals=4)

# Chinmay's Algorithm:

## Convex Program for CBCP and DBCP Equilibria:

In [17]:
## Steps
# 1: Define variables
# 2: Define objective
# 3: Define constraints
# 4: Define problem
# 5: Solve problem
# 6: Extract values

In [18]:
# num_edges = 8
# od_to_edges_array = np.array([[0, 0], [0, 3], [0, 7], [1, 5], [1, 7], [3, 3], [3, 6]])

# od_to_edges_list_full = [list(range(od_to_edges_array[od, 0], od_to_edges_array[od, 1] + 1)) \
#                       for od in range(od_to_edges_array.shape[0])]

# edges_to_od = [[od for od in range(od_to_edges_array.shape[0]) if e in od_to_edges_list_full[od]] \
#             for e in range(num_edges)]

# print(od_to_edges_list_full)
# print()
# print(edges_to_od)

In [38]:

# Below: For quartic latency functions:
# Latency Function: a_4 x^4 + a_3 x^3 + a_2 x^2 + a_1 x + a_0

def solve_CBCP_direct(T, num_edges, num_gp_lanes, tau, B, od_to_edges_array, \
                      demand_array, VoT_array, num_el, coeff_input):
    
#     print("tau (in solve_CBCP_direct):", tau)
    
    coeff = coeff_from_input(coeff_input, num_edges, num_gp_lanes)
    
    assert demand_array.shape == VoT_array.shape, "demand_array and VoT_array should have identical shape."
    assert np.all(demand_array > 0.0), "Each entry of demand_array must be strictly positive."
    assert np.all(tau >= 0.0), "Each entry of tau must be non-negative."
    assert num_el <= demand_array.shape[1], "num_el, the number of eligible income groups, should not exceed \
                                            demand_array.shape[1], which should equal the number of income groups."
    
    num_groups = demand_array.shape[1]
    
    ## Variable indices:
    # y indices: (od, income group, edge, "lane", time)
    # y indices: (od, income group, edge, "lane", time)

    el_indices = list(range(num_el))
    in_indices = list(range(num_el, num_groups))
    group_indices = list(range(num_groups))
    
    od_to_edges_list_full = [list(range(int(od_to_edges_array[od, 0]), int(od_to_edges_array[od, 1]) + 1)) \
                          for od in range(od_to_edges_array.shape[0])]
    
    edge_to_ods = [[od for od in range(od_to_edges_array.shape[0]) if e in od_to_edges_list_full[od]] \
                   for e in range(num_edges)]
    
    num_od = len(od_to_edges_list_full)
    
    # Variables:
    y = {}
    for od in range(num_od):
        for e in od_to_edges_list_full[od]:
            for t in range(T):
                for g in el_indices:
                    for k in [0, 1, 2]:
                        y[(od, g, e, k, t)] = cp.Variable(1)
                for g in in_indices:
                    for k in [0, 1]:
                        y[(od, g, e, k, t)] = cp.Variable(1)
    
    x = {}
    for e in range(num_edges):
        for k in [0, 1]:
            for t in range(T):
                x[(e, k, t)] = cp.Variable(1)

    # Objective:
    func = 0.0
    for e in range(num_edges):
        for k in range(2):
            for t in range(T):
                func += coeff[0, e, k] * x[e, k, t] \
                        + 0.5 * coeff[1, e, k] * cp.square(cp.maximum(x[e, k, t] - coeff[2, e, k], 0))
    
    print("VoT_array.shape:", VoT_array.shape)
    print("el_indices:", el_indices)
    print("in_indices:", in_indices)
    
    for od in range(num_od):
        for e in od_to_edges_list_full[od]:
            for t in range(T):
                for g in el_indices:
                    func += tau[e, t] * y[(od, g, e, 1, t)] / VoT_array[od, g]
                for g in in_indices:
                    func += tau[e, t] * y[(od, g, e, 0, t)] / VoT_array[od, g]

    objective = cp.Minimize(func)
    
    # Constraints:
    constraints = []
    
    constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) \
                    for e in od_to_edges_list_full[od] for g in el_indices  \
                    for k in [0, 1, 2] for t in range(T)]
    constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) \
                    for e in od_to_edges_list_full[od] for g in in_indices  \
                    for k in [0, 1] for t in range(T)]
    constraints += [x[(e, k, t)] >= 0.0 for e in range(num_edges) \
                    for k in [0, 1] for t in range(T)]
    
#     print()
#     print("demand_array:", demand_array)
#     print()
#     print("B:", B)
#     print()
#     print("el_indices:", el_indices)
#     print()
#     print("in_indices:", in_indices)
#     print()
#     print("edge_to_ods:", edge_to_ods)
#     print()
#     print("od_to_edges_list_full:", od_to_edges_list_full)
#     print()
    
    od_g_e_t_list = []

    for e in range(num_edges):
        for t in range(T):
            
            ## Edge contraints:
            constraints += [sum( y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)] for od in edge_to_ods[e] for g in el_indices) \
                                + sum( y[(od, g, e, 0, t)] for od in edge_to_ods[e] for g in in_indices ) \
                                == x[(e, 0, t)] ]
            constraints += [sum( y[(od, g, e, 2, t)] for od in edge_to_ods[e] for g in el_indices) \
                                + sum( y[(od, g, e, 1, t)] for od in edge_to_ods[e] for g in in_indices ) \
                                == x[(e, 1, t)] ]

            ## Group flow constraints:
            for od in edge_to_ods[e]:
                for g in el_indices:
#                     print("(od, g, e, t):", od, g, e, t)
                    assert (od, g, e, t) not in od_g_e_t_list, "Each (od, g, e, t) should occur only once."
                    od_g_e_t_list += [(od, g, e, t)]
        
#                     if (od, g, e, t) == (0, 0, 0, 0):
#                     if (e, t) == (2, 0) or (2, 1) or (2, 3):
#                         constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1, 2]) == demand_array[od, g]]
                    
#                 for g in in_indices:
# #                     print("(od, g, e, t):", od, g, e, t)
#                     assert (od, g, e, t) not in od_g_e_t_list, "Each (od, g, e, t) should occur only once."
#                     od_g_e_t_list += [(od, g, e, t)]
                    
#                     constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1]) == demand_array[od, g]]

            constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1, 2]) == demand_array[od, g] \
                            for od in edge_to_ods[e] for g in el_indices]
            constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1]) == demand_array[od, g] \
                            for od in edge_to_ods[e] for g in in_indices]
    
    constraints += [sum(y[(od, g, e, 0, t)] * tau[e, t] for e in od_to_edges_list_full[od] for t in range(T)) \
                    <= B * demand_array[od, g] for od in range(num_od) for g in el_indices]
    
    # Problem:
    prob = cp.Problem(objective, constraints)
    
    # Solve:
#     result = prob.solve(solver = cp.MOSEK, verbose = True)
    result = prob.solve(solver = cp.MOSEK)
    
#     for variable in prob.variables():
#         print("Variable %s" % variable.name())
    
    assert prob.status != "infeasible", "problem.status should not be infeasible."
    assert prob.status != "unbounded", "problem.status should not be unbounded."
    print()
    print("prob.status:", prob.status)

    # Extract Values:
    y_values = {}
    for e in range(num_edges):
        for od in edge_to_ods[e]:
            for t in range(T):
                for g in el_indices:
                    for k in [0, 1, 2]:
#                         print("(od, g, e, k, t):", od, g, e, k, t)
#                         print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
                        y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)
                for g in in_indices:
                    for k in [0, 1]:
#                         print("(od, g, e, k, t):", od, g, e, k, t)
#                         print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
                        y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)

    return y_values


## Implement Zeroth-order Gradient Descent:

In [20]:
## Functions defined above:

# def proj_tau_B(T, num_edges, tau, B, od_to_edges_list_full, tau_max = 1.0, B_max = 1.0):

# def solve_CBCP_direct(T, num_edges, num_gp_lanes, tau, B, od_to_edges_array, \
#                       demand_array, VoT_array, num_el, coeff_input):

#     def welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, tau, \
#                 demand_array, VoT_array, num_el, od_to_edges_array, y, \
#                 coeff_input = np.array([0.0, 0.0, 0.0, 0.0, 1.0])):

## <font color='red'>To edit below</font> 

In [50]:
time_1 = time.time()

T = 5

num_el = 2

assert demand_array.shape == VoT_array.shape, "demand_array and VoT_array should have the same shape."

num_ods = demand_array.shape[0]
group_indices = list(range(demand_array.shape[1]))
num_edges = 6
num_gp_lanes = 3

# tau = np.array([0.2, 0.6, 0.5, 0.8, 0.4, 0.5])
# assert tau.shape[0] == num_edges, "tau must have a num_edges number of entries."

# B = 0.8
a = np.array([1E-3, 1E-3, 0.0, 0.0, 0.0])

error_bound = 1E-2
diffs_num_cols = 5

# lambda_E, lambda_R, lambda_I = 1.0, 1.0, 1.0
lambda_E, lambda_R, lambda_I = 1.0, 1.0, 0.1
# lambda_E, lambda_R, lambda_I = 1.0, 1.5, 1.0

demand_edges_array = np.zeros(num_edges)

# demand_array_temp = np.ones(demand_array.shape)
# demand_array_temp = np.random.uniform(low=0.05, high=0.5, size=demand_array.shape)
demand_array_temp = demand_array


# TODO: Modify coeff_input:

## coeff_input: const, slope, x-location of kink

coeff_input = np.array([19.4, 0.1256, 0.786*1650]).reshape((3, 1)) @ np.ones((1, num_edges))
# coeff_input = np.zeros((3, num_edges))

counter = 0
for city in dict_latency_params.keys():
    coeff_input[0, counter] = dict_latency_params[city]["Flow (at bend)"]
    coeff_input[1, counter] = dict_latency_params[city]["Slope (after bend)"]
    coeff_input[2, counter] = dict_latency_params[city]["Time (at bend)"]
    
    counter += 1


# def solve_CBCP_direct(T, num_edges, num_gp_lanes, tau, B, od_to_edges_array, \
#                       demand_array, VoT_array, num_el, coeff_input):
# Already defined: T, num_edges, num_gp_lanes, tau, B, od_to_edges_array, \
#                       demand_array, VoT_array

    
# print("od_to_edges_array[1, 0]:", int(od_to_edges_array[1, 0]))
# print("range(od_to_edges_array[1, 0], od_to_edges_array[1, 1] + 1):",\
#      range(int(od_to_edges_array[1, 0]), int(od_to_edges_array[1, 1]) + 1))

od_to_edges_list_full = [list(range(int(od_to_edges_array[od, 0]), int(od_to_edges_array[od, 1]) + 1)) \
                         for od in range(num_ods)]
edge_to_ods = [[od for od in range(od_to_edges_array.shape[0]) if e in od_to_edges_list_full[od]] \
               for e in range(num_edges)]
for e in range(num_edges):
    demand_edges_array[e] = sum([np.sum(demand_array_temp[od, :]) for od in range(num_ods) \
                                 if od in edge_to_ods[e]])

tau_max_from_latency = np.zeros(num_edges)
for e in range(num_edges):
    tau_max_from_latency[e] = latency_max(demand_edges_array[e], coeff_input[:, e]) / (1 + num_gp_lanes)

tau_max_monetary_value = 15.0
# fraction_tau_max = tau_max_monetary_value / np.max(tau_max_from_latency)

tau_upper_limit = np.minimum(tau_max_from_latency, tau_max_monetary_value)
B_upper_limit = max([sum(tau_upper_limit[e] for e in od_to_edges_list_full[od]) \
                     for od in range(len(od_to_edges_list_full))])

d = num_edges * T + 1
# num_iters = 1000
num_iters = 100
# num_iters = 10
# num_iters = 3
tau = np.zeros((num_edges, T, num_iters))
tau_perturbed = np.zeros((num_edges, T, num_iters))

B = np.zeros(num_iters)
B_perturbed = np.zeros(num_iters)
delta = np.zeros(num_iters)
eta = np.zeros(num_iters)
eta_bar = 0.1
delta_bar = 1.0

welfare_list = []

fraction_tau_max = 0.5
fraction_B_max = 0.2

# tau[:, :, 0] = fraction_tau_max * tau_upper_limit.reshape((num_edges, 1)) @ np.ones((1, T))
tau_upper_limit_array = tau_upper_limit.reshape((num_edges, 1)) @ np.ones((1, T))
tau[:, :, 0] = np.random.triangular(np.zeros((num_edges, T)), tau_upper_limit_array * fraction_tau_max, tau_upper_limit_array)
# B[0] = fraction_B_max * B_upper_limit
B[0] = np.random.triangular(0.0, fraction_B_max * B_upper_limit, B_upper_limit)

od_to_edges_list_full = [list(range(int(od_to_edges_array[od, 0]), int(od_to_edges_array[od, 1]) + 1 )) \
                         for od in range(od_to_edges_array.shape[0])]

# print("od_to_edges_list_full:", od_to_edges_list_full)
# print("tau_max_from_latency:", tau_max_from_latency)
# print("B_max", B_max)

for i in range(num_iters-1):
    
    print()
    print("Iter:", i)
    
    eta[i] = eta_bar * (i+1)**(-1/2) * d**(-1)
    delta[i] = delta_bar * (i+1)**(-1/4) * d**(-1/2)
    w_i_unnormalized = np.random.randn(d)
    w_i = w_i_unnormalized / np.linalg.norm(w_i_unnormalized)
#     print("w_i:", w_i)
    tau_perturbed[:, :, i] = tau[:, :, i] + delta[i] * w_i[:-1].reshape((num_edges, T))
    B_perturbed[i] = B[i] + delta[i] * w_i[-1]
    
#     if tau_perturbed[:, :, i] < B_perturbed[i] or tau_perturbed[:, :, i] < 0 or B_perturbed[i] < 0:

    print("i:\n", i)
    print("tau[:, :, i], before projection:\n", tau[:, :, i])
    print("B[i], before projection:\n", B[i])
    print("tau_perturbed[:, :, i], before projection:\n", tau_perturbed[:, :, i])
    print("B_perturbed[i], before projection:\n", B_perturbed[i])

    tau_perturbed[:, :, i], B_perturbed[i] = proj_tau_B(T, num_edges, tau_perturbed[:, :, i], B_perturbed[i], \
                                                        od_to_edges_list_full, \
                                                        tau_max = tau_upper_limit, B_max = B_upper_limit)
    
    print("tau[:, :, i], after projection:\n", tau[:, :, i])
    print("B[i], after projection:\n", B[i])
    print("tau_perturbed[:, :, i], after projection:\n", tau_perturbed[:, :, i])
    print("B_perturbed[i], after projection:\n", B_perturbed[i])

    # TODO: Remove "network":

    y_values = solve_CBCP_direct(T, num_edges, num_gp_lanes, \
                                 tau[:, :, i], B[i], od_to_edges_array, \
                                 demand_array_temp, VoT_array, num_el, coeff_input)

    y_perturbed_values = solve_CBCP_direct(T, num_edges, num_gp_lanes, \
                                           tau_perturbed[:, :, i], B_perturbed[i], od_to_edges_array, \
                                           demand_array_temp, VoT_array, num_el, coeff_input)
    
#     def welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, tau, \
#                 demand_array, VoT_array, num_el, od_to_edges_array, y, \
#                 coeff_input):

    welfare, obj_E, obj_R, obj_I = welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, \
                                               tau[:, :, i], demand_array_temp, VoT_array, num_el, od_to_edges_array, \
                                               y = y_values, coeff_input = coeff_input)
    
    welfare_perturbed, obj_E_perturbed, obj_R_perturbed, obj_I_perturbed \
        = welfare_obj(T, num_edges, num_gp_lanes, lambda_E, lambda_R, lambda_I, \
                      tau_perturbed[:, :, i], demand_array_temp, VoT_array, num_el, od_to_edges_array, \
                      y = y_perturbed_values, coeff_input = coeff_input)
    
    print("welfare:", welfare)
    print("obj_E:", obj_E)
    print("obj_R:", obj_R)
    print("obj_I:", obj_I)
    print()
    print("welfare_perturbed:", welfare_perturbed)
    print("obj_E_perturbed:", obj_E_perturbed)
    print("obj_R_perturbed:", obj_R_perturbed)
    print("obj_I_perturbed:", obj_I_perturbed)
    print()
    
    welfare_list.append(welfare)
    
    tau[:, :, i+1] = tau[:, :, i] - eta[i] * (d/delta[i]) * w_i[:-1].reshape((num_edges, T)) \
                        * (welfare_perturbed - welfare)
    
    B[i+1] = B[i] - eta[i] * (d/delta[i]) * w_i[-1] * (welfare_perturbed - welfare)
    
    print("tau[:, :, i+1], before projection:\n", tau[:, :, i+1])
    print("B[i+1], before projection:\n", B[i+1])
    
    tau[:, :, i+1], B[i+1] = proj_tau_B(T, num_edges, tau[:, :, i+1], B[i+1], od_to_edges_list_full, \
                                        tau_max = tau_upper_limit, B_max = B_upper_limit)
    
    print("tau[:, :, i+1], after projection:\n", tau[:, :, i+1])
    print("B[i+1], after projection:\n", B[i+1])
    
    if i >= diffs_num_cols + 2:
        tau_diffs = np.max(np.absolute(tau[:, :, i-diffs_num_cols : i-1] - tau[:, :, i-diffs_num_cols+1 : i]), axis = 2)
        B_diffs = B[i-diffs_num_cols : i-1] - B[i-diffs_num_cols+1 : i]
        
#         print("tau[:, :, 0:10]:", tau[:, :, 0:10])
#         print("B[0:10]:", B[0:10])
        print("tau_diffs:\n", tau_diffs)
        print("B_diffs:\n", B_diffs)
        print()
        
        if max(np.max(np.absolute(tau_diffs)), np.max(np.absolute(B_diffs))) < error_bound:
            print("Within error bound.")
            break

time_2 = time.time()

min_welfare = min(welfare_list)
argmin_welfare_list = welfare_list.index(min(welfare_list))
argmin_tau = tau[:, :, argmin_welfare_list]
argmin_B = B[argmin_welfare_list]

print()
print("Time:", time_2 - time_1)




Iter: 0
i:
 0
tau[:, :, i], before projection:
 [[11.74923727  9.1825627   6.15674294 10.83951736  8.42859416]
 [ 6.2209673   7.95526907  6.91369599  3.21744099  9.32511161]
 [ 7.28743928  3.92256416 10.31483326  8.07246858  6.6167598 ]
 [ 6.06733921 14.08139188  8.23881784  2.39820954  4.5726885 ]
 [ 8.79140477  8.5748968  11.85509324  2.23467306  0.77785157]
 [ 9.11553656 10.43026792  4.18795786  4.20385509  4.93852472]]
B[i], before projection:
 36.40499933474742
tau_perturbed[:, :, i], before projection:
 [[11.71560604  9.20105326  6.10187542 10.87407985  8.39745789]
 [ 6.21019968  7.97671111  6.86807769  3.1951523   9.31228643]
 [ 7.31444214  3.87883039 10.31997107  8.05189103  6.55509449]
 [ 6.05648372 14.12067862  8.25589655  2.42414645  4.53969005]
 [ 8.84856819  8.58704763 11.82731861  2.26099345  0.83377952]
 [ 9.12201937 10.41685135  4.19505083  4.24530844  4.92240224]]
B_perturbed[i], before projection:
 36.442757856403006

B: 36.442757856403006
B_max_wrt_tau_feas: 32.9016


prob.status: optimal
VoT_array.shape: (19, 5)
el_indices: [0, 1]
in_indices: [2, 3, 4]

prob.status: optimal

in_indices: [2, 3, 4]
el_indices: [0, 1]


in_indices: [2, 3, 4]
el_indices: [0, 1]

welfare: 19541456.95676985
obj_E: 1450099.0727128494
obj_R: 0.4202699172035357
obj_I: 180913583.0432692

welfare_perturbed: 19541455.432452057
obj_E_perturbed: 1450099.2810857035
obj_R_perturbed: 1.9780761872497419
obj_I_perturbed: 180913581.29442537

tau[:, :, i+1], before projection:
 [[11.1007052   9.41318843  5.67179191 11.15058237  8.02998281]
 [ 5.86420563  7.53799608  6.4371097   3.00758999  8.91739859]
 [ 7.72353908  3.6926859  10.22725985  7.79843114  5.96892384]
 [ 5.85493342 14.5092908   8.15709198  2.64834493  4.43084054]
 [ 9.24561555  8.8630246  11.30746269  2.58410849  1.48130396]
 [ 9.31967842 10.45940079  4.46908152  4.12168153  4.90765613]]
B[i+1], before projection:
 32.73323465538233

B: 32.73323465538233
B_max_wrt_tau_feas: 33.27749839529168
B_feas: 32.73323465538233

tau[


prob.status: optimal
VoT_array.shape: (19, 5)
el_indices: [0, 1]
in_indices: [2, 3, 4]

prob.status: optimal

in_indices: [2, 3, 4]
el_indices: [0, 1]


in_indices: [2, 3, 4]
el_indices: [0, 1]

welfare: 19541455.5745464
obj_E: 1450099.4537798408
obj_R: 1.7049768447331617
obj_I: 180913578.25743404

welfare_perturbed: 19541455.832440972
obj_E_perturbed: 1450099.4393843396
obj_R_perturbed: 1.429945433294332
obj_I_perturbed: 180913578.23002064

tau[:, :, i+1], before projection:
 [[10.91952803  9.52777517  5.84449587 11.15059287  8.33240393]
 [ 6.06809278  7.21751285  6.77870799  2.87467889  8.79411117]
 [ 7.61237479  3.97231459 10.02443843  7.86417932  5.74786612]
 [ 5.91347569 14.8076084   8.23105272  2.50010313  4.5703344 ]
 [ 9.32752303  9.14274843 11.11483827  2.68199577  1.57190499]
 [ 9.60291757 10.28587932  4.09268235  4.20116304  4.80020886]]
B[i+1], before projection:
 32.58287537420729

B: 32.58287537420729
B_max_wrt_tau_feas: 32.98285113662123
B_feas: 32.58287537420729

tau[:


prob.status: optimal
VoT_array.shape: (19, 5)
el_indices: [0, 1]
in_indices: [2, 3, 4]

prob.status: optimal

in_indices: [2, 3, 4]
el_indices: [0, 1]


in_indices: [2, 3, 4]
el_indices: [0, 1]

welfare: 19541456.579362016
obj_E: 1450099.2800656823
obj_R: 0.7893221330155432
obj_I: 180913580.8861847

welfare_perturbed: 19541455.80489842
obj_E_perturbed: 1450099.3808226897
obj_R_perturbed: 1.5656472843737501
obj_I_perturbed: 180913579.89723012

tau[:, :, i+1], before projection:
 [[10.88649133  9.45947619  5.86518889 11.04821639  8.41292054]
 [ 6.00489216  7.1371959   6.78230183  2.95688277  8.8272226 ]
 [ 7.68069885  4.09596188 10.03393451  7.89207522  5.76234354]
 [ 5.893474   14.69227245  8.11162129  2.418335    4.51323334]
 [ 9.47903536  9.18436354 11.03629929  2.53229866  1.58840994]
 [ 9.53679902 10.397383    4.02767889  4.14074042  4.79076674]]
B[i+1], before projection:
 32.5655431659935

B: 32.5655431659935
B_max_wrt_tau_feas: 32.89336806815866
B_feas: 32.5655431659935

tau[:, 


prob.status: optimal
VoT_array.shape: (19, 5)
el_indices: [0, 1]
in_indices: [2, 3, 4]

prob.status: optimal

in_indices: [2, 3, 4]
el_indices: [0, 1]


in_indices: [2, 3, 4]
el_indices: [0, 1]

welfare: 19541455.126259863
obj_E: 1450099.5940687966
obj_R: 1.9608609379216508
obj_I: 180913574.93052006

welfare_perturbed: 19541456.144143224
obj_E_perturbed: 1450099.4239600594
obj_R_perturbed: 0.9792432609937954
obj_I_perturbed: 180913576.99426425

tau[:, :, i+1], before projection:
 [[10.87944451  9.5123397   5.93031822 11.00390381  8.46589264]
 [ 6.06201461  6.88473381  6.82181292  3.02421461  8.77021962]
 [ 7.69195968  4.21873823 10.15699564  7.77975011  5.96367742]
 [ 5.82668685 14.77424437  8.14719474  2.46850835  4.56180357]
 [ 9.36029357  9.12042113 11.06250899  2.58090254  1.53785264]
 [ 9.3880937  10.42517321  4.00355653  4.14849454  4.77697829]]
B[i+1], before projection:
 32.56224018086331

B: 32.56224018086331
B_max_wrt_tau_feas: 32.74229627081806
B_feas: 32.56224018086331

ta

...


prob.status: optimal
VoT_array.shape: (19, 5)
el_indices: [0, 1]
in_indices: [2, 3, 4]

prob.status: optimal

in_indices: [2, 3, 4]
el_indices: [0, 1]


in_indices: [2, 3, 4]
el_indices: [0, 1]

welfare: 19541455.857532725
obj_E: 1450099.7459553431
obj_R: 0.6234219552735076
obj_I: 180913567.34999335

welfare_perturbed: 19541452.201131787
obj_E_perturbed: 1450100.1983923693
obj_R_perturbed: 4.409220122898459
obj_I_perturbed: 180913564.1195954

tau[:, :, i+1], before projection:
 [[11.0826301   9.45912715  6.18531329 12.02161197  8.30799061]
 [ 6.22249339  7.35129657  6.47274304  3.28094583  8.70207125]
 [ 7.44256546  3.78853891  9.63221558  8.83715338  6.3124362 ]
 [ 5.94034051 14.67284005  7.85683874  2.11911325  4.9169235 ]
 [10.17104142  9.02071626 10.75652618  2.11924637  2.09206157]
 [ 8.58811638 11.12824229  4.77066583  3.97493167  5.70683336]]
B[i+1], before projection:
 31.724728361236096

B: 31.724728361236096
B_max_wrt_tau_feas: 34.1687895400601
B_feas: 31.724728361236096

ta


prob.status: optimal
VoT_array.shape: (19, 5)
el_indices: [0, 1]
in_indices: [2, 3, 4]

prob.status: optimal

in_indices: [2, 3, 4]
el_indices: [0, 1]


in_indices: [2, 3, 4]
el_indices: [0, 1]

welfare: 19541451.717489514
obj_E: 1450100.1869048788
obj_R: 5.026198691726085
obj_I: 180913565.56783324

welfare_perturbed: 19541454.589211665
obj_E_perturbed: 1450099.807814298
obj_R_perturbed: 2.0827380443040098
obj_I_perturbed: 180913568.64135408

tau[:, :, i+1], before projection:
 [[10.93757058  9.37528026  6.08940441 11.98651635  8.13965516]
 [ 6.20067815  7.3671929   6.42751151  3.33301749  8.8700515 ]
 [ 7.4303908   3.86031603  9.59448123  8.76867093  6.30844272]
 [ 5.75412711 14.73312509  7.93207576  2.01390537  4.82425154]
 [10.05206941  8.88550767 10.73950709  2.08301605  2.04292561]
 [ 8.78197376 11.18370882  4.78306607  4.15449405  5.73163833]]
B[i+1], before projection:
 31.548529174180334

B: 31.548529174180334
B_max_wrt_tau_feas: 34.63488103257126
B_feas: 31.548529174180334

t


prob.status: optimal
VoT_array.shape: (19, 5)
el_indices: [0, 1]
in_indices: [2, 3, 4]

prob.status: optimal

in_indices: [2, 3, 4]
el_indices: [0, 1]


in_indices: [2, 3, 4]
el_indices: [0, 1]

welfare: 19541455.148147188
obj_E: 1450099.709243983
obj_R: 1.6166475408991026
obj_I: 180913570.55550745

welfare_perturbed: 19541455.79426395
obj_E_perturbed: 1450099.604676626
obj_R_perturbed: 0.9529313053160511
obj_I_perturbed: 180913571.4251863

tau[:, :, i+1], before projection:
 [[10.9399934   9.44918105  6.05680088 11.94345096  8.19099299]
 [ 6.24847418  7.35600585  6.43844496  3.35733343  8.90365324]
 [ 7.41213768  3.79526737  9.60324838  8.73385086  6.24309884]
 [ 5.76281434 14.69469858  7.89551046  2.02325364  4.84592202]
 [ 9.98188185  8.88106577 10.74546065  2.11894678  2.04861846]
 [ 8.79009668 11.15017094  4.75658648  4.13763915  5.69222502]]
B[i+1], before projection:
 31.55280190695876

B: 31.55280190695876
B_max_wrt_tau_feas: 34.52671827293408
B_feas: 31.55280190695876

tau[:,


prob.status: optimal
VoT_array.shape: (19, 5)
el_indices: [0, 1]
in_indices: [2, 3, 4]

prob.status: optimal

in_indices: [2, 3, 4]
el_indices: [0, 1]


in_indices: [2, 3, 4]
el_indices: [0, 1]

welfare: 19541451.879390966
obj_E: 1450100.153653996
obj_R: 4.926429218853634
obj_I: 180913566.52166188

welfare_perturbed: 19541454.882558323
obj_E_perturbed: 1450099.7900620427
obj_R_perturbed: 1.8162682328045605
obj_I_perturbed: 180913569.08764514

tau[:, :, i+1], before projection:
 [[11.18514875  9.49530442  6.10288365 12.11862611  8.17709757]
 [ 6.18996214  7.364496    6.39030676  3.32776331  8.92425394]
 [ 7.35343266  3.82242225  9.46427419  8.81810401  6.14588809]
 [ 5.57496916 14.92787649  7.81763254  2.08453749  5.0854761 ]
 [ 9.94424548  8.94726442 10.60875545  1.92972449  2.17748266]
 [ 8.79856752 11.15779899  4.71952786  4.15097883  5.70012802]]
B[i+1], before projection:
 31.45252079845883

B: 31.45252079845883
B_max_wrt_tau_feas: 34.52700122809891
B_feas: 31.45252079845883

tau[


prob.status: optimal
VoT_array.shape: (19, 5)
el_indices: [0, 1]
in_indices: [2, 3, 4]

prob.status: optimal

in_indices: [2, 3, 4]
el_indices: [0, 1]


in_indices: [2, 3, 4]
el_indices: [0, 1]

welfare: 19541454.910437092
obj_E: 1450099.8562139934
obj_R: 1.7456936729390389
obj_I: 180913567.9991677

welfare_perturbed: 19541455.195476476
obj_E_perturbed: 1450099.8102730196
obj_R_perturbed: 1.5014220579501492
obj_I_perturbed: 180913568.86625513

tau[:, :, i+1], before projection:
 [[11.31148546  9.45462287  6.03147668 11.99787667  8.1474425 ]
 [ 6.21357076  7.36104841  6.31806923  3.33197411  8.91299981]
 [ 7.35308369  3.87099047  9.3431945   8.80509595  6.04340013]
 [ 5.78543887 14.92521302  7.81295441  2.13527861  5.07292019]
 [10.00217818  9.03706434 10.72789088  1.98294411  2.27820021]
 [ 8.79018861 11.23565646  4.59751317  4.10011344  5.84766545]]
B[i+1], before projection:
 31.450917910546483

B: 31.450917910546483
B_max_wrt_tau_feas: 34.57113713446686
B_feas: 31.450917910546483



In [51]:
# tau_upper_limit
argmin_welfare_list

24

In [46]:
# y_perturbed_values = solve_CBCP_direct(T, num_edges, num_gp_lanes, \
#                                            tau_perturbed[:, :, i], B_perturbed[i], od_to_edges_array, \
#                                            demand_array_temp, VoT_array, num_el, coeff_input)

# T
# num_edges
# num_gp_lanes
# B_perturbed[i]
# coeff_input
# od_to_edges_array
# demand_array_temp
# VoT_array
# num_el
# coeff_input



In [47]:
# solve_CBCP_direct(T, num_edges, num_gp_lanes, \
#                                            tau_perturbed[:, :, i], B_perturbed[i], od_to_edges_array, \
#                                            demand_array_temp, VoT_array, num_el, coeff_input)

In [ ]:
# tau[:, :, 0]

# y_values

tau_upper_limit

In [ ]:
# demand_array_temp
B_upper_limit

In [ ]:
# np.minimum(np.array([1, 2]), 0.5)

In [ ]:
tau_upper_limit

# Compute Cost Metrics

# Testing solve CBCP direct with a small example:

In [ ]:
# od_to_edges_array = np.array([[0, 0], [0, 1], [1, 1], [1, 2]])
# od_to_edges_list_full = [[0], [0, 1], [1], [1, 2]]
# edge_to_ods = [[0, 1], [1, 2, 3], [3]]

# num_od = 3
# num_edges = 3
# num_gp_lanes = 3
# num_groups = 5
# T = 5
# el_indices = [0, 1]
# in_indices = [2, 3, 4]
# B = 3
# tau = np.zeros((num_edges, T))

# for e in range(num_edges):
#     for t in range(T):
#         tau[e, t] = 1.5 + 0.5 * e + 0.2 * t

# demand_array = np.zeros((num_od, num_groups))
# VoT_array = np.zeros((num_od, num_groups))

# for od in range(num_od):
#     for g in range(num_groups):
#         demand_array[od, g] = 1.0 + od + 0.1 * g
#         VoT_array[od, g] = 2.0 + od + 0.1 * g
        
# coeff_input = np.array([1, 1, 0, 0, 0])
# latency_params_length = coeff_input.shape[0]

# ex_to_gp_multiplier = np.array([1/num_gp_lanes**p for p in range(latency_params_length)]).reshape((latency_params_length, 1)) \
#                         @ np.ones((1, num_edges))
# a = np.zeros((latency_params_length, num_edges, 2))
# coeff[:, :, 0] = coeff_input.reshape((latency_params_length, 1)) @ np.ones((1, num_edges))
# coeff[:, :, 1] = coeff[:, :, 0] * ex_to_gp_multiplier


# y_vals = solve_CBCP_direct(T, num_edges, num_gp_lanes, tau, B, od_to_edges_array, \
#                            demand_array, VoT_array, num_el, coeff_input)


# # y = {}
# # for od in range(num_od):
# #     for e in od_to_edges_list_full[od]:
# #         for t in range(T):
# #             for g in el_indices:
# #                 for k in [0, 1, 2]:
# #                     y[(od, g, e, k, t)] = cp.Variable(1)
# #             for g in in_indices:
# #                 for k in [0, 1]:
# #                     y[(od, g, e, k, t)] = cp.Variable(1)

# # x = {}
# # for e in range(num_edges):
# #     for k in [0, 1]:
# #         for t in range(T):
# #             x[(e, k, t)] = cp.Variable(1)

# # # Objective:
# # func = 0.0
# # for e in range(num_edges):

# # for od in range(num_od):
# #     for e in od_to_edges_list_full[od]:
# #         for t in range(T):
# #             for g in el_indices:
# #                 func += tau[e, t] * y[(od, g, e, 1, t)] / VoT_array[od, g]
# #             for g in in_indices:
# #                 func += tau[e, t] * y[(od, g, e, 0, t)] / VoT_array[od, g]

# # objective = cp.Minimize(func)

# # # Constraints:
# # constraints = []

# # constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) \
# #                 for e in od_to_edges_list_full[od] for g in el_indices  \
# #                 for k in [0, 1, 2] for t in range(T)]
# # constraints += [y[(od, g, e, k, t)] >= 0.0 for od in range(num_od) \
# #                 for e in od_to_edges_list_full[od] for g in in_indices  \
# #                 for k in [0, 1] for t in range(T)]


# # for e in range(num_edges):
# #     for t in range(T):
# # #         print("e:", e)
# # #         print("edge_to_ods[e]:", edge_to_ods[e])

# #         ## Edge contraints:
# #         constraints += [sum( y[(od, g, e, 0, t)] + y[(od, g, e, 1, t)] for od in edge_to_ods[e] for g in el_indices) \
# #                             + sum( y[(od, g, e, 0, t)] for od in edge_to_ods[e] for g in in_indices ) \
# #                             == x[(e, 0, t)] ]
# #         constraints += [sum( y[(od, g, e, 2, t)] for od in edge_to_ods[e] for g in el_indices) \
# #                             + sum( y[(od, g, e, 1, t)] for od in edge_to_ods[e] for g in in_indices ) \
# #                             == x[(e, 1, t)] ]

# #         ## Group flow constraints:
# #         constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1, 2]) == demand_array[od, g] \
# #                         for od in edge_to_ods[e] for g in el_indices]
# #         constraints += [sum(y[(od, g, e, k, t)] for k in [0, 1]) == demand_array[od, g] \
# #                         for od in edge_to_ods[e] for g in in_indices]

# # constraints += [sum(y[(od, g, e, 0, t)] * tau[e, t] for e in od_to_edges_list_full[od] for t in range(T)) \
# #                 <= B * demand_array[od, g] for od in range(num_od) for g in el_indices]

# # # Problem:
# # prob = cp.Problem(objective, constraints)

# # # Solve:
# # result = prob.solve()


# # assert prob.status != "infeasible", "problem.status should not be infeasible."
# # assert prob.status != "unbounded", "problem.status should not be unbounded."
# # print()
# # print("prob.status:", prob.status)

# # # Extract Values:
# # y_values = {}
# # for e in range(num_edges):
# #     for od in edge_to_ods[e]:
# #         for t in range(T):
# #             for g in el_indices:
# #                 for k in [0, 1, 2]:
# #                     print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
# #                     y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)
# #             for g in in_indices:
# #                 for k in [0, 1]:
# #                     print("y[(od, g, e, k, t)].value:", y[(od, g, e, k, t)].value)
# #                     y_values[(od, g, e, k, t)] = max(y[(od, g, e, k, t)].value[0], 0.0)




In [ ]:
# demand_edges_array

In [ ]:
# np.sum(demand_array[3:16, :])/5

In [ ]:
print("first(welfare_list):", welfare_list[0])
print("min(welfare_list):", min(welfare_list))
print("max(welfare_list):", max(welfare_list))
print("argmin_tau:", argmin_tau)
print("argmin_B:", argmin_B)

## Test:

## <font color='red'>Colored Font Titles</font> 

# Scratch Work:

In [ ]:
x = cp.Variable(2)
y = cp.Variable(2)
v_fixed = np.array([0, 1])
objective = cp.Minimize(cp.sum_squares(x - y) + cp.sum_squares(x - v_fixed))
constraints = []
prob = cp.Problem(objective, constraints)

# The optimal objective value is returned by `prob.solve()`.
result = prob.solve()
# The optimal value for x is stored in `x.value`.
print("x.value:", x.value)
print("y.value:", y.value)
print()


## CVXPY can handle 4d arrays:

In [ ]:

I, J, K, L = 2, 3, 4, 5

# Variables:
x_test = {}
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                x_test[(i, j, k, ell)] = cp.Variable(1)
            
# Objective:
func = 0.0
func += cp.sum([x_test[(i, j, k, ell)]**2 for i in range(I) for j in range(J) \
                for k in range(K) for ell in range(L)])
            
objective = cp.Minimize(func)

# Constraints:
constraints = []

for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                constraints += [cp.sum([x_test[(i, j, k, ell)] for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L) ]) == 1.0]
                constraints += [x_test[(i, j, k, ell)] >= 0.0 for i in range(I) for j in range(J) \
                                        for k in range(K) for ell in range(L)]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
for i in range(I):
    for j in range(J):
        for k in range(K):
            for ell in range(L):
                print("i, j, k, ell:", i, j, k, ell)
                print("x_test[(i,j,k, ell)].value:", x_test[(i, j, k, ell)].value)


## Linear Approximation for Latency Function:

In [ ]:
# Variables:
v = cp.Variable(1)
            
# Objective:
func = v - 1 + cp.square(cp.maximum(v-1, 0))
objective = cp.Minimize(func)

# Constraints:
constraints = [-3.0 <= v, v <= 3.0]

# Solve problem:
prob = cp.Problem(objective, constraints)
result = prob.solve()

# Print solution:
print("v.value:", v.value)
